In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
pip install requests beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [23]:
import requests
from bs4 import BeautifulSoup

base_url = "https://www.overleaf.com"
learn_url = base_url + "/learn"

# Request main Learn page
headers = {"User-Agent": "Mozilla/5.0"}
def get_guide_links():
    """Fetch all the guide links from the main Overleaf Learn page."""
    learn_url = base_url + "/learn"
    response = requests.get(learn_url, headers={"User-Agent": "Mozilla/5.0"})
    
    if response.status_code != 200:
        print(f"Failed to retrieve the page. Status code: {response.status_code}")
        return []

    soup = BeautifulSoup(response.text, "html.parser")

    links = []
    for a in soup.select("a[href^='/learn/']"):  # All internal /learn/ links
        full_link = base_url + a["href"]
        links.append(full_link)

    if not links:
        print("No links found.")
    return links

In [29]:
def get_content_div(soup):
    options = [
        lambda s: s.find("div", {"class": "mw-parser-output"}),
        lambda s: s.find("main"),
        lambda s: s.find("div", recursive=False)
    ]
    for option in options:
        content = option(soup)
        if content and content.find(["p", "h2", "pre", "ul", "ol"]):
            return content
    return None

def scrape_guide(url):
    """Scrape headings, paragraphs, lists, and code blocks from a guide page."""
    response = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
    soup = BeautifulSoup(response.text, "html.parser")

    
    # content_div = get_content_div(soup)
    # if not content_div:
    #     print("behf")
    #     return None

    guide_data = {
        "title": soup.find("h1").get_text(strip=True) if soup.find("h1") else "Untitled",
        "url": url,
        "content": []  # Flat structure for content
    }

    # Loop through the content and collect all parts (text, lists, code)
    for element in soup.find_all(["h2", "h3", "p", "ul", "ol", "pre"]):
        if element.name in ["h2", "h3"]:
            heading_text = element.get_text(strip=True)
            if heading_text:  # Only append if the heading is not empty
                guide_data["content"].append({"type": "heading", "data": heading_text})
        elif element.name == "p":
            paragraph_text = element.get_text(strip=True)
            if paragraph_text:  # Only append if the paragraph is not empty
                guide_data["content"].append({"type": "text", "data": paragraph_text})
        elif element.name in ["ul", "ol"]:
            list_items = [li.get_text(strip=True) for li in element.find_all("li")]
            if list_items:  # Only append if the list has items
                guide_data["content"].append({"type": "list", "data": list_items})
        elif element.name == "pre":
            code_text = element.get_text(strip=True)
            if code_text:  # Only append if the code block is not empty
                guide_data["content"].append({"type": "code", "data": code_text})

    return guide_data

In [30]:
links = get_guide_links()

all_guides = []
for link in tqdm(links, desc="Scraping Overleaf Guides"):
    guide_data = scrape_guide(link)
    if guide_data:
        all_guides.append(guide_data)

with open("overleaf_guides.json", "w", encoding="utf-8") as f:
    json.dump(all_guides, f, ensure_ascii=False, indent=4)

print(f"✅ Scraped {len(all_guides)} guides and saved to overleaf_guides.json")

Scraping Overleaf Guides: 100%|██████████| 151/151 [00:12<00:00, 12.45it/s]


✅ Scraped 151 guides and saved to overleaf_guides.json
